# Benchmarking on STM32-Cube.ai



In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd
import numpy as np

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.wandb import wandb_model_DB, get_model_DB_run_id_from_architecture, get_architecture_from_model_DB_run_id

import wandb
%load_ext autoreload

In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
models_dir = initialize()

In [4]:
automated = False

global model_name

model_name = "mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV1"



models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

c:\tiny_mlc\tiny_cnn\models


In [5]:
models_stm32_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_stm32_benchmark")
models_stm32_benchmark_path

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV1/mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV1_stm32_benchmark')

In [6]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32tflm $models_tflite_path

In [7]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32tfl.exe

In [8]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32ai 

In [9]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32ai validate --name network -m $models_tflite_path --type tflite --compression none --verbosity 1 --workspace C:\Users\Susanne\AppData\Local\Temp\mxAI_workspace1067734059012002168110269332871471 --output $models_stm32_benchmark_path --allocate-inputs --allocate-outputs --mode stm32 --desc 115200 --with-report

# Helper functions

In [10]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [11]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [12]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [13]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

In [14]:
models_stm32_benchmark_file = models_dir.joinpath(model_name, "network_output", "network_validate_report.txt")
models_stm32_benchmark_file
models_stm32_benchmark_file_INT8 = models_dir.joinpath(model_name, "network_output", "network_validate_report_INT8.txt")
models_stm32_benchmark_file_INT8

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV1/network_output/network_validate_report_INT8.txt')

In [15]:
def clean_benchmark_file(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                #line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [16]:
#! explorer $models_path

In [17]:
! code $models_stm32_benchmark_file

In [18]:
! code $models_stm32_benchmark_file_INT8

In [19]:
clean_lines = clean_benchmark_file(models_stm32_benchmark_file_INT8)
#clean_lines

In [20]:
def get_index_of_benchmark_lines(lines):
    model_lines_dict = {}
    for i, line in enumerate(lines):
        if "model name            :" in line:
            model_lines_dict["model_name"] = i
        elif "m_id   layer (type,original)" in line:
            model_lines_dict["m_id"] = i

        elif "model/c-model:" in line:
            model_lines_dict["model/c-model"]=i
            
        elif "activations size" in line:
            model_lines_dict["activations_size"] = i
        elif "weights size" in line:
            model_lines_dict["weights_size"] = i
        elif "macc                  :" in line:
            model_lines_dict["macc"] = i
        elif "duration        :" in line:
            model_lines_dict["duration"] = i
        elif "cycles/MACC     :" in line:
            model_lines_dict["cycles/MACC"] = i
        elif "counter         :" in line:
            model_lines_dict["counter"] = i

        elif "C-Arrays" in line:
            model_lines_dict["c-arrays"] = i
        elif "C-Layers" in line:
            model_lines_dict["c-layers"] = i
        elif "Inference time per node" in line:
            model_lines_dict["inference_times"] = i

        elif "Statistic per tensor" in line:
            model_lines_dict["statistics"] = i
        elif "Number of operations per c-layer" in line:
            model_lines_dict["number_of_operations"] = i
        elif "Number of operation types" in line:
            model_lines_dict["number_of_operation_types"] = i
        elif "Complexity report" in line:
            model_lines_dict["complexity_report"] = i
        elif "Evaluation report " in line:
            model_lines_dict["evaluation_report"] = i
    #print(model_lines_dict)
    return model_lines_dict


In [21]:
model_lines_dict = get_index_of_benchmark_lines(clean_lines)
model_lines_dict

{'m_id': 33,
 'model/c-model': 413,
 'model_name': 419,
 'activations_size': 423,
 'weights_size': 424,
 'macc': 425,
 'c-arrays': 429,
 'c-layers': 685,
 'duration': 1268,
 'cycles/MACC': 1270,
 'counter': 1271,
 'inference_times': 1276,
 'statistics': 1405,
 'number_of_operations': 1437,
 'number_of_operation_types': 1565,
 'complexity_report': 1575,
 'evaluation_report': 1699}

In [22]:
# model_name
name_line = clean_lines[model_lines_dict["model_name"]]
name = name_line.split(":")[1].split()[0].strip()
name

'mobilenetv3smallnsq_0_25_96_c3_o2_se4l128_mv1_int8'

In [23]:
suffix=""
inference_stats = {}


In [24]:
if "int8" in name:
    suffix = "_INT8"

In [25]:
# weights_size
activations_line = clean_lines[model_lines_dict["activations_size"]]
activations = activations_line.split(":")[1].split()[0].strip()
inference_stats[f"stm32_activations_b{suffix}"]= activations
activations

'95432'

In [26]:
# weights_size
weights_line = clean_lines[model_lines_dict["weights_size"]]
weights = weights_line.split(":")[1].split()[0].strip()
inference_stats[f"stm32_weights_b{suffix}"]= weights
weights

'72480'

In [27]:
# MACC
macc_line = clean_lines[model_lines_dict["macc"]]
macc = macc_line.split(":")[1].strip()
inference_stats[f"stm32_macc_b{suffix}"]= macc
macc

'6711320'

In [28]:
# cycles/MACC

try:
    cycles_line = clean_lines[model_lines_dict["cycles/MACC"]]
    cycles_line = cycles_line.split(":")[1].strip()
    cycles = float(cycles_line)
    inference_stats[f"stm32_cycles/macc_b{suffix}"]= cycles
    cycles
except:
    print("cylces/MACC not reported")

In [29]:
# inference time
duration_line = clean_lines[model_lines_dict["duration"]]
duration_line = duration_line.split(":")[1].strip().split()[0].replace("ms", "")# .map(float)
duration = float(duration_line)
inference_stats[f"stm32_inference_ms{suffix}"]= duration
duration

120.23

In [30]:
inference_stats

{'stm32_activations_b_INT8': '95432',
 'stm32_weights_b_INT8': '72480',
 'stm32_macc_b_INT8': '6711320',
 'stm32_cycles/macc_b_INT8': 3.87,
 'stm32_inference_ms_INT8': 120.23}

In [31]:
m_id_lines = clean_lines[model_lines_dict["m_id"]: model_lines_dict["model/c-model"]]
m_id_lines

['m_id   layer (type,original)                                                               oshape                 param/size           macc                                                        connected to   | c_size          c_macc              c_type                               \n',
 '------ ----------------------------------------------------------------------------------- ---------------------- --------------- --------- ------------------------------------------------------------------- --- --------------- ------------------- ------------------------------------ \n',
 '0      serving_default_input_20 (Input, )                                                  [b:1,h:96,w:96,c:3]                                                                                                    | +496(+100.0%)   +995,344(+100.0%)   optimized_conv2d_oi8[0]              \n',
 '       conv2d_0 (Conv2D, CONV_2D)                                                          [b:1,h:48,w:48,c:16]   448/496 

In [32]:
suffix

'_INT8'

In [33]:
m_indices = [0, 7, 64, 87, 105, 114, 157, 160, 181, 200, None]
m_indices_INT8 = [0, 6, 68, 113, 129, 139,  211, 227, 247, None]

if suffix =="_INT8":
    m_indices = m_indices_INT8

split_list =[]
for line in m_id_lines[0 :-3]:
    if "---" in line:
        pass
    else:
        split_list.append([line[m_indices[i]:m_indices[i+1]].strip() for i in range(len(m_indices)-1)])
#split_list

m_id_df = pd.DataFrame(split_list)
col_names = m_id_df.iloc[0]
m_id_df = m_id_df[1:]
m_id_df.columns = col_names

m_id_df["m_id"] = m_id_df["m_id"].str.strip()
# m_id_df["m_id"].loc[m_id_df["m_id"]==""] = np.NAN
# m_id_df["m_id"] = m_id_df["m_id"].fillna(method="ffill")
m_id_df

,m_id,"layer (type,original)",oshape,param/size,macc,connected to |,c_size,c_macc,c_type
1,0,"serving_default_input_20 (Input, )","[b:1,h:96,w:96,c:3]",,,|,+496(+100.0%),"+995,344(+100.0%)",optimized_conv2d_oi8[0]
2,,"conv2d_0 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:16]",448/496,"995,344",serving_default_input_20 |,-496(-100.0%),"-995,344(-100.0%)",
3,1,"mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y (Placeholder, )",[],1/1,,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[1]
4,,"eltwise_1 (Eltwise, MUL)","[b:1,h:48,w:48,c:16]",,"36,864",conv2d_0 |,,"-36,864(-100.0%)",
5,,,,,,mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y |,,,
...,...,...,...,...,...,...,...,...,...
237,131,"eltwise_131 (Eltwise, MUL)","[b:1,h:3,w:3,c:576]",,"5,184",conv2d_126 |,,,eltwise_integer_oi8[115]
238,,,,,,nl_130 |,,,
239,132,"pool_132 (Pool, AVERAGE_POOL_2D)","[b:1,h:1,w:1,c:576]",,"5,184",eltwise_131 |,,,pool_oi8[116]
240,133,"reshape_133 (Reshape, RESHAPE)","[b:1,c:576]",,,pool_132 |,,,


In [34]:
m_id_df["macc"].loc[m_id_df["macc"]==""] = np.NAN
m_id_df["macc"] = m_id_df["macc"].str.replace(",", "")
m_id_df["macc"].fillna(0, inplace=True)
m_id_df["macc"] = m_id_df["macc"].astype(int)
m_id_df


,m_id,"layer (type,original)",oshape,param/size,macc,connected to |,c_size,c_macc,c_type
1,0,"serving_default_input_20 (Input, )","[b:1,h:96,w:96,c:3]",,0,|,+496(+100.0%),"+995,344(+100.0%)",optimized_conv2d_oi8[0]
2,,"conv2d_0 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:16]",448/496,995344,serving_default_input_20 |,-496(-100.0%),"-995,344(-100.0%)",
3,1,"mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y (Placeholder, )",[],1/1,0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[1]
4,,"eltwise_1 (Eltwise, MUL)","[b:1,h:48,w:48,c:16]",,36864,conv2d_0 |,,"-36,864(-100.0%)",
5,,,,,0,mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y |,,,
...,...,...,...,...,...,...,...,...,...
237,131,"eltwise_131 (Eltwise, MUL)","[b:1,h:3,w:3,c:576]",,5184,conv2d_126 |,,,eltwise_integer_oi8[115]
238,,,,,0,nl_130 |,,,
239,132,"pool_132 (Pool, AVERAGE_POOL_2D)","[b:1,h:1,w:1,c:576]",,5184,eltwise_131 |,,,pool_oi8[116]
240,133,"reshape_133 (Reshape, RESHAPE)","[b:1,c:576]",,0,pool_132 |,,,


In [35]:
# mask = m_id_df["m_id"].duplicated()
# s1 = '.' + m_id_df[mask].groupby('m_id').cumcount().add(1).astype(str)
# m_id_df.loc[mask, 'm_id'] += s1
# m_id_df


In [36]:
m_id_df["m_id"].unique()

array(['0', '', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99',
       '100', '101', '102', '103', '104', '105', '106', '107', '108',
       '109', '110', '111', '112', '113', '114', '115', '116', '117',
       '118', '119', '120', '121', '122', '123', '124', '125', '126',
       '127', '128', '129', '130', '131', '132', '133', '134'],
      dtype=object)

In [37]:
m_id_df.drop(m_id_df[m_id_df["layer (type,original)"]==""].index, inplace=True)

In [38]:
#m_id_df["layer (type,original)"].unique()

In [39]:
layer_split_df = m_id_df["layer (type,original)"].str.split(" \(", expand=True)
layer_split_df[1] = layer_split_df[1].str.replace("\)", "")
layer_split_df

C:\Users\Susanne\AppData\Local\Temp\ipykernel_23148\1069216173.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  layer_split_df[1] = layer_split_df[1].str.replace("\)", "")


,0,1
1,serving_default_input_20,"Input,"
2,conv2d_0,"Conv2D, CONV_2D"
3,mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y,"Placeholder,"
4,eltwise_1,"Eltwise, MUL"
6,mobilenetv3smallNSQ_tf_math_add_18_Add_y,"Placeholder,"
...,...,...
236,nl_130,"Nonlinearity, RELU"
237,eltwise_131,"Eltwise, MUL"
239,pool_132,"Pool, AVERAGE_POOL_2D"
240,reshape_133,"Reshape, RESHAPE"


In [40]:
m_id_df["layer_name"] = layer_split_df[0]
m_id_df["type"] = layer_split_df[1]
m_id_df

,m_id,"layer (type,original)",oshape,param/size,macc,connected to |,c_size,c_macc,c_type,layer_name,type
1,0,"serving_default_input_20 (Input, )","[b:1,h:96,w:96,c:3]",,0,|,+496(+100.0%),"+995,344(+100.0%)",optimized_conv2d_oi8[0],serving_default_input_20,"Input,"
2,,"conv2d_0 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:16]",448/496,995344,serving_default_input_20 |,-496(-100.0%),"-995,344(-100.0%)",,conv2d_0,"Conv2D, CONV_2D"
3,1,"mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y (Placeholder, )",[],1/1,0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[1],mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y,"Placeholder,"
4,,"eltwise_1 (Eltwise, MUL)","[b:1,h:48,w:48,c:16]",,36864,conv2d_0 |,,"-36,864(-100.0%)",,eltwise_1,"Eltwise, MUL"
6,2,"mobilenetv3smallNSQ_tf_math_add_18_Add_y (Placeholder, )",[],1/1,0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[2],mobilenetv3smallNSQ_tf_math_add_18_Add_y,"Placeholder,"
...,...,...,...,...,...,...,...,...,...,...,...
236,130,"nl_130 (Nonlinearity, RELU)","[b:1,h:3,w:3,c:576]",,5184,eltwise_129 |,,"-5,184(-100.0%)",,nl_130,"Nonlinearity, RELU"
237,131,"eltwise_131 (Eltwise, MUL)","[b:1,h:3,w:3,c:576]",,5184,conv2d_126 |,,,eltwise_integer_oi8[115],eltwise_131,"Eltwise, MUL"
239,132,"pool_132 (Pool, AVERAGE_POOL_2D)","[b:1,h:1,w:1,c:576]",,5184,eltwise_131 |,,,pool_oi8[116],pool_132,"Pool, AVERAGE_POOL_2D"
240,133,"reshape_133 (Reshape, RESHAPE)","[b:1,c:576]",,0,pool_132 |,,,,reshape_133,"Reshape, RESHAPE"


In [41]:
param_split_df = m_id_df["param/size"].str.split("/", expand=True)
param_split_df[0].loc[param_split_df[0]==""] = np.NAN

param_split_df[0] = param_split_df[0].str.replace(",", "")
param_split_df[1] = param_split_df[1].str.replace(",", "")
param_split_df.fillna(0, inplace=True)
param_split_df[0] = param_split_df[0].astype(int)
param_split_df[1] = param_split_df[1].astype(int)

param_split_df

,0,1
1,0,0
2,448,496
3,1,1
4,0,0
6,1,1
...,...,...
236,0,0
237,0,0
239,0,0
240,0,0


In [42]:
m_id_df["params"] = param_split_df[0]
m_id_df["size"] = param_split_df[1]
m_id_df

,m_id,"layer (type,original)",oshape,param/size,macc,connected to |,c_size,c_macc,c_type,layer_name,type,params,size
1,0,"serving_default_input_20 (Input, )","[b:1,h:96,w:96,c:3]",,0,|,+496(+100.0%),"+995,344(+100.0%)",optimized_conv2d_oi8[0],serving_default_input_20,"Input,",0,0
2,,"conv2d_0 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:16]",448/496,995344,serving_default_input_20 |,-496(-100.0%),"-995,344(-100.0%)",,conv2d_0,"Conv2D, CONV_2D",448,496
3,1,"mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y (Placeholder, )",[],1/1,0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[1],mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y,"Placeholder,",1,1
4,,"eltwise_1 (Eltwise, MUL)","[b:1,h:48,w:48,c:16]",,36864,conv2d_0 |,,"-36,864(-100.0%)",,eltwise_1,"Eltwise, MUL",0,0
6,2,"mobilenetv3smallNSQ_tf_math_add_18_Add_y (Placeholder, )",[],1/1,0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[2],mobilenetv3smallNSQ_tf_math_add_18_Add_y,"Placeholder,",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,130,"nl_130 (Nonlinearity, RELU)","[b:1,h:3,w:3,c:576]",,5184,eltwise_129 |,,"-5,184(-100.0%)",,nl_130,"Nonlinearity, RELU",0,0
237,131,"eltwise_131 (Eltwise, MUL)","[b:1,h:3,w:3,c:576]",,5184,conv2d_126 |,,,eltwise_integer_oi8[115],eltwise_131,"Eltwise, MUL",0,0
239,132,"pool_132 (Pool, AVERAGE_POOL_2D)","[b:1,h:1,w:1,c:576]",,5184,eltwise_131 |,,,pool_oi8[116],pool_132,"Pool, AVERAGE_POOL_2D",0,0
240,133,"reshape_133 (Reshape, RESHAPE)","[b:1,c:576]",,0,pool_132 |,,,,reshape_133,"Reshape, RESHAPE",0,0


In [43]:
m_id_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 1 to 241
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   m_id                   162 non-null    object
 1   layer (type,original)  162 non-null    object
 2   oshape                 162 non-null    object
 3   param/size             162 non-null    object
 4   macc                   162 non-null    int32 
 5   connected to   |       162 non-null    object
 6   c_size                 162 non-null    object
 7   c_macc                 162 non-null    object
 8   c_type                 162 non-null    object
 9   layer_name             162 non-null    object
 10  type                   144 non-null    object
 11  params                 162 non-null    int32 
 12  size                   162 non-null    int32 
dtypes: int32(3), object(10)
memory usage: 15.8+ KB


In [44]:
#m_id_df.to_csv("m_id_df.csv")

In [45]:
# m_id_df_slimm = m_id_df.copy()
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Input, "].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Placeholder, "].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Pad, PAD"].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type==""].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, HARD_SWISH"].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, DEPTHWISE_CONV_2D"].index, inplace=True)
# m_id_df_slimm.dropna(subset=["type"], inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, CONV_2D"].index, inplace=True)
# #m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type is None].index, inplace=True)
# m_id_df_slimm.reset_index(inplace=True)

# m_id_df_slimm.index +=1
# m_id_df_slimm

In [46]:
#m_id_df_slimm.type.unique()

In [47]:
#m_id_df_slimm.info()

In [48]:
# #m_id_df["layer (type,original)"] = m_id_df["layer (type,original)"].replace(", ", ",")
# split_df = m_id_df["layer (type,original)"].str.split(" ",expand=True)
# split_df["(type,original)"] = split_df[1] + split_df[2]
# split_df.drop([1], inplace=True)
# #split_df.dropna(inplace=True)
# split_df

# Getting inference times per layer

In [49]:
inference_lines = clean_lines[model_lines_dict["inference_times"]: model_lines_dict["statistics"]]
inference_lines

['   Inference time per node\n',
 '   ---------------------------------------------------------------------------------------------\n',
 '   c_id    m_id   type                 dur (ms)     %         counters            name          \n',
 '   ---------------------------------------------------------------------------------------------\n',
 '   0       0      Conv2dPool (0x109)       12.772     10.6%   [      2,758,665]   ai_node_0     \n',
 '   1       1      EltwiseInt (0x114)        1.232      1.0%   [        266,026]   ai_node_1     \n',
 '   2       2      EltwiseInt (0x114)        6.230      5.2%   [      1,345,671]   ai_node_2     \n',
 '   3       3      EltwiseInt (0x114)        1.044      0.9%   [        225,575]   ai_node_3     \n',
 '   4       5      EltwiseInt (0x114)        1.606      1.3%   [        346,954]   ai_node_4     \n',
 '   5       6      Conv2D (0x103)            4.951      4.1%   [      1,069,358]   ai_node_5     \n',
 '   6       7      Pad (0x116)         

In [50]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [59]:
indices = [0, 8, 15,36,49, 60,79, None]
split_list =[]
for line in inference_lines[2 :-3]:
    if "---" in line:
        pass
    else:
        split_list.append([line[indices[i]:indices[i+1]].strip() for i in range(len(indices)-1)])

inference_df = pd.DataFrame(split_list)
col_names = inference_df.iloc[0]
inference_df = inference_df[1:]
inference_df.columns = col_names
inference_df["c_id"] = inference_df["c_id"].str.strip()
inference_df["dur (ms)"] = inference_df["dur (ms)"].astype(float)
inference_df["%"] = inference_df["%"].apply(string_percent_to_float)
inference_df["counters"] = inference_df["counters"].str.replace("[", "")
inference_df["counters"] = inference_df["counters"].str.replace("]", "")
inference_df["counters"] = inference_df["counters"].str.replace(",", "").astype(int)
inference_df.rename(columns={"name": "node_name", "type": "node_type"}, errors="raise", inplace=True)
#print(col_names)
inference_df

C:\Users\Susanne\AppData\Local\Temp\ipykernel_23148\1061571099.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  inference_df["counters"] = inference_df["counters"].str.replace("[", "")
C:\Users\Susanne\AppData\Local\Temp\ipykernel_23148\1061571099.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  inference_df["counters"] = inference_df["counters"].str.replace("]", "")


,c_id,m_id,node_type,dur (ms),%,counters,node_name
1,0,0,Conv2dPool (0x109),12.772,10.6,2758665,ai_node_0
2,1,1,EltwiseInt (0x114),1.232,1.0,266026,ai_node_1
3,2,2,EltwiseInt (0x114),6.230,5.2,1345671,ai_node_2
4,3,3,EltwiseInt (0x114),1.044,0.9,225575,ai_node_3
5,4,5,EltwiseInt (0x114),1.606,1.3,346954,ai_node_4
...,...,...,...,...,...,...,...
117,116,132,Pool (0x10b),0.408,0.3,88140,ai_node_116
118,117,134,Dense (0x104),0.023,0.0,4942,ai_node_117
119,118,134,NL (0x107),0.001,0.0,295,ai_node_118
120,119,135,NL (0x107),0.004,0.0,905,ai_node_119


In [60]:
inference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 1 to 121
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   c_id       121 non-null    object 
 1   m_id       121 non-null    object 
 2   node_type  121 non-null    object 
 3   dur (ms)   121 non-null    float64
 4   %          121 non-null    float64
 5   counters   121 non-null    int32  
 6   node_name  121 non-null    object 
dtypes: float64(2), int32(1), object(4)
memory usage: 6.3+ KB


In [61]:
#inference_df.to_csv("inference_df.csv")

In [99]:
merged_df = pd.merge(inference_df,m_id_df,on='m_id', how="outer")#inference_df.join(m_id_df_slimm, on="m_id", how="left")
merged_df.head(20)

,c_id,m_id,node_type,dur (ms),%,counters,node_name,"layer (type,original)",oshape,param/size,macc,connected to |,c_size,c_macc,c_type,layer_name,type,params,size
0,0,0,Conv2dPool (0x109),12.772,10.6,2758665.0,ai_node_0,"serving_default_input_20 (Input, )","[b:1,h:96,w:96,c:3]",,0.0,|,+496(+100.0%),"+995,344(+100.0%)",optimized_conv2d_oi8[0],serving_default_input_20,"Input,",0.0,0.0
1,1,1,EltwiseInt (0x114),1.232,1.0,266026.0,ai_node_1,"mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y (Placeholder, )",[],1/1,0.0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[1],mobilenetv3smallNSQ_tf_math_multiply_18_Mul_y,"Placeholder,",1.0,1.0
2,2,2,EltwiseInt (0x114),6.230,5.2,1345671.0,ai_node_2,"mobilenetv3smallNSQ_tf_math_add_18_Add_y (Placeholder, )",[],1/1,0.0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[2],mobilenetv3smallNSQ_tf_math_add_18_Add_y,"Placeholder,",1.0,1.0
3,3,3,EltwiseInt (0x114),1.044,0.9,225575.0,ai_node_3,mobilenetv3smallNSQ_tf_clip_by_value_18_clip_by_value_Minimum,"_y (Placeholder, ) []",1/1,0.0,|,+3(+300.0%),"+36,864(+100.0%)",eltwise_integer_oi8[3],mobilenetv3smallNSQ_tf_clip_by_value_18_clip_by_value_Minimum,None,1.0,1.0
4,4,5,EltwiseInt (0x114),1.606,1.3,346954.0,ai_node_4,"eltwise_5 (Eltwise, MUL)","[b:1,h:48,w:48,c:16]",,36864.0,conv2d_0 |,,,eltwise_integer_oi8[4],eltwise_5,"Eltwise, MUL",0.0,0.0
5,5,6,Conv2D (0x103),4.951,4.1,1069358.0,ai_node_5,"conv2d_6 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:16]",272/320,589840.0,eltwise_5 |,,,conv2d_oi8[5],conv2d_6,"Conv2D, CONV_2D",272.0,320.0
6,6,7,Pad (0x116),0.132,0.1,28514.0,ai_node_6,"conv2d_7 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:16]",160/208,331792.0,nl_6_nl |,,,"pad_oi8/conv2d_oi8[6, 7]",conv2d_7,"Conv2D, DEPTHWISE_CONV_2D",160.0,208.0
7,7,7,Conv2D (0x103),6.672,5.5,1441100.0,ai_node_7,"conv2d_7 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:16]",160/208,331792.0,nl_6_nl |,,,"pad_oi8/conv2d_oi8[6, 7]",conv2d_7,"Conv2D, DEPTHWISE_CONV_2D",160.0,208.0
8,8,8,Conv2D (0x103),2.429,2.0,524682.0,ai_node_8,"conv2d_8 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",136/160,294920.0,nl_7_nl |,,,conv2d_oi8[8],conv2d_8,"Conv2D, CONV_2D",136.0,160.0
9,9,9,Conv2D (0x103),10.947,9.1,2364513.0,ai_node_9,"conv2d_9 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:40]",360/480,737320.0,conv2d_8 |,,,conv2d_oi8[9],conv2d_9,"Conv2D, CONV_2D",360.0,480.0


In [100]:
merged_df.drop(merged_df[merged_df.type=="Nonlinearity, HARD_SWISH"].index, inplace=True)
merged_df.drop(merged_df[merged_df.type=="Nonlinearity, DEPTHWISE_CONV_2D"].index, inplace=True)

merged_df.drop(merged_df[merged_df.type=="Nonlinearity, CONV_2D"].index, inplace=True)
merged_df.drop(merged_df[merged_df.type=="Eltwise, MINIMUM"].index, inplace=True)

merged_df.drop(merged_df[merged_df.type=="Nonlinearity, RELU"].index, inplace=True)
#merged_df.drop(merged_df[merged_df["layer (type,original)"=="Nonlinearity, RELU"].index, inplace=True)
merged_df.dropna(subset=["layer (type,original)"], inplace=True)
merged_df.dropna(subset=["c_id"], inplace=True)
merged_df["type"].fillna("", inplace=True)
merged_df.drop_duplicates(subset=["layer (type,original)"], keep="first", inplace=True)
#merged_df.drop("index", axis=1, inplace=True)
merged_df.reset_index(inplace=True, drop=False)
merged_df.rename(columns={"index": "_index"}, errors="raise", inplace=True)


In [101]:
merged_df.columns

Index(['_index', 'c_id', 'm_id', 'node_type', 'dur (ms)', '%', 'counters',
       'node_name', 'layer (type,original)', 'oshape', 'param/size', 'macc',
       'connected to   |', 'c_size', 'c_macc', 'c_type', 'layer_name', 'type',
       'params', 'size'],
      dtype='object', name=0)

In [90]:
merged_df.isnull().sum()

0
c_id                     0
m_id                     0
node_type                0
dur (ms)                 0
%                        0
counters                 0
node_name                0
layer (type,original)    0
oshape                   0
param/size               0
macc                     0
connected to   |         0
c_size                   0
c_macc                   0
c_type                   0
layer_name               0
type                     0
params                   0
size                     0
dtype: int64

In [ ]:
# combined_df = m_id_df_slimm.copy()
# combined_df["c_type"] = inference_df["type"]
# combined_df["dur (ms)"] = inference_df["dur (ms)"]
# combined_df["counters"] = inference_df["counters"]
# combined_df["node_name"] = inference_df["node_name"]
# combined_df

In [ ]:
#combined_df[["m_id", "layer_name", "type_x", "type_y", "dur (ms)"]]

In [ ]:
#inference_df["c_id"].unique()

# Logging to wandb

In [102]:
# Generate run ids
#id = wandb.wandb.sdk.lib.runid.generate_id()

id = get_model_DB_run_id_from_architecture(model_name)
PROJECT = "model_DB"

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        id = id, 
        resume="allow",
        )


# # Create a table
stm32_inference_df_table = wandb.Table(dataframe=inference_df)
stm32_int8_table = wandb.Table(dataframe= merged_df)

run.log({"stm32_inference_times_int8": stm32_inference_df_table})
run.log({"stm32_inference_int8": stm32_int8_table})

run.log(inference_stats)



wandb.finish()

stm32_cycles/macc_b_INT8,▁
stm32_inference_ms_INT8,▁
allocate_tensors_ms_%,1.379
allocate_tensors_ms_avg,0.116
allocate_tensors_ms_first,0.116
arena_size,506848
first_inference_us,12403
inference_avg_us,1893.24
init_us,183602
initialization_ms,183.602
model_size_MB,0.17528
